In [4]:
import pandas as pd
import numpy as np
import OpenBlender
import json
%matplotlib inline
import matplotlib

In [5]:
token = '61521f809516292e82e5264aLvxmIcHpcGcHzuZ4a0e2hplW40Tn24'
action = 'API_getObservationsFromDataset'
# ANCHOR: 'Lit vs USD'
parameters = {
    'token' : token,
    'id_dataset' : '5d4c94f59516290b02fecb02',
    'date_filter':{"start_date" : "2020-01-01",
                   "end_date" : "2021-09-19"}
}
df = pd.read_json(json.dumps(OpenBlender.call(action, parameters)['sample']),
                  convert_dates=False, convert_axes=False).sort_values('timestamp', ascending=False)
df.reset_index(drop=True, inplace=True)
df['date'] = [OpenBlender.unixToDate(ts, timezone = 'GMT') for ts in df.timestamp]
df = df.drop('timestamp', axis = 1)

Task ID: '615262a80895fafb4a9d8d4b'.
Total estimated consumption: 500.06 processing units.
Task confirmed. Starting download..
100.0 % completed.


In [6]:
print(df.shape)

(586, 7)


In [7]:
df.head()
df.to_csv(r'Lit_prices.csv')

In [8]:
df['log_diff'] = np.log(df['close']) - np.log(df['open'])
df

,volume,market_cap,high,low,close,open,date,log_diff
0,372409.22400,372409.22400,185.80,177.90,181.40,180.00,18-09-2021 17:00:00,0.007748
1,372409.22400,372409.22400,185.80,177.90,181.40,180.00,18-09-2021 17:00:00,0.007748
2,566767.06900,566767.06900,190.50,177.50,180.00,185.20,17-09-2021 17:00:00,-0.028479
3,566767.06900,566767.06900,190.50,177.50,180.00,185.20,17-09-2021 17:00:00,-0.028479
4,935286.23100,935286.23100,195.40,181.50,185.20,189.10,16-09-2021 17:00:00,-0.020840
...,...,...,...,...,...,...,...,...
581,291964.84958,291964.84958,44.61,42.60,43.30,42.79,05-01-2020 17:00:00,0.011848
582,248814.62818,248814.62818,42.90,41.77,42.78,42.23,04-01-2020 17:00:00,0.012940
583,426771.00676,426771.00676,42.39,38.77,42.23,39.44,03-01-2020 17:00:00,0.068350
584,194486.21570,194486.21570,41.74,39.17,39.43,41.58,02-01-2020 17:00:00,-0.053092


In [9]:
df['target'] = [1 if log_diff > 0 else 0 for log_diff in df['log_diff']]
df
df.to_csv(r'Lit_prices_log.csv')

In [10]:
format = '%d-%m-%Y %H:%M:%S'
timezone = 'GMT'
df['timestamp'] = OpenBlender.dateToUnix(df['date'], date_format = format, timezone = timezone)
df = df[['date', 'timestamp', 'close', 'target']]
df.to_csv(r'Lit_prices_target.csv')

In [14]:
text_filter = {'name' : 'bitcoin',
               'match_ngrams': ['positive', 'buy', 'bitcoin',
                                'bull', 'boost','optimistic','strength','rally','negative',
                                'loss', 'drop', 'plummet', 'sell', 'fundraising',
                                'dump','slip','bear']}
# We need to add the 'id_dataset' and the 'feature' name we want.
blend_source = {
                'id_dataset':'5ea209c495162936348f13eb',
                'feature' : 'text',
                'filter_text' : text_filter
            }
# Now, let's 'timeBlend' it to our dataset
df_blend = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list'
                                  ).add_prefix('VEC.')
df_anchor = pd.concat([df, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)
df_anchor
df_anchor.to_csv(r'crypto_id_dataset_news.csv')

Task ID: '615263c50895fafb4a9d8d4c'.
Total estimated consumption: 2675.2 processing units.
Task confirmed. Starting download..
100%


In [15]:
# We add the ngrams to match on a 'positive' feature.
positive_filter = {'name' : 'positive',
                   'match_ngrams': ['positive', 'buy',
                                    'bull', 'boost','optimistic','strength','rally']}
blend_source = {
                'id_dataset':'5ea209c495162936348f13eb',
                'feature' : 'text',
                'filter_text' : positive_filter
            }
df_blend = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df = pd.concat([df, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)
# And now the negatives
negative_filter = {'name' : 'negative',
                   'match_ngrams': ['negative','loss', 'drop', 'plummet', 'sell', 'fundraising','dump','slip','bear']}
blend_source = {
                'id_dataset':'5ea209c495162936348f13eb',
                'feature' : 'text',
                'filter_text' : negative_filter
            }
df_blend = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals', #closest_observation
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df = pd.concat([df, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)
df.to_csv(r'Lite_ngrams.csv')

Task ID: '615263cc0895fafb4a9d8d4c'.
Total estimated consumption: 2675.2 processing units.
Task confirmed. Starting download..
100%
Task ID: '615263d00895fafb4a9d8d4c'.
Total estimated consumption: 2675.2 processing units.
Task confirmed. Starting download..
100%


In [16]:
features = ['target', 'CRYPTOCURR.text_COUNT_last1days:positive', 'CRYPTOCURR.text_COUNT_last1days:negative']
df[features].corr()['target']
df_anchor = df[features].corr()['target']
print(df_anchor)
df_anchor.to_csv(r'Lite_correlation.csv')

target                                      1.000000
CRYPTOCURR.text_COUNT_last1days:positive    0.008536
CRYPTOCURR.text_COUNT_last1days:positive    0.008536
CRYPTOCURR.text_COUNT_last1days:negative   -0.001385
CRYPTOCURR.text_COUNT_last1days:negative   -0.001385
Name: target, dtype: float64
